# Assignment 3

## Instructions - Read this first!

This is an individual homework assignment. This means that:

- You may discuss the problems in this assignment with other students in this course and your instructor/TA, but YOUR WORK MUST BE YOUR OWN.
- Do not show other students code or your own work on this assignment.
- You may consult external references, but not actively receive help from individuals not involved in this course.
- Cite all references outside of the course you used, including conversations with other students which were helpful. (This helps us give credit where it is due!). All references must use a commonly accepted reference format, for example, APA or IEEE (or another citation style of your choice).

If any of these rules seem ambiguous, please check with with your instructor for help interpreting them.

We suggest completing this assignment using the provided notebook. Each question should be answered using a SQL query (or combination or SQL queries) unless the text indicates that you may (or should) do something else. You may submit your queries embedded in Python, using SQLAlchemy or the MySQL Connector, or as plain text in Markdown.

## When you submit your work

Your submission will be graded manually. To ensure that everything goes smoothly, please follow these instructions to prepare your notebook for submission to the D2L Dropbox for Assignment 3:

- Please remove any print statments used to test your work (you can comment them out)
- Please provide your solutions where asked; please do not alter any other parts of this notebook.
- If you need to add cells to test your code please move them to the end of the notebook before submission- or you may included your commented out answers and tests in the cells provided

## Introduction

In this assignment, you will continue to practice and extend your SQL skills, and compare your work to MongoDB.

We will be using two datasets from Open Data Calgary. The Census by Community 2019 dataset shows results of the 2019 Civic Census. This is a separate process from that used by Statistics Canada to produce the national census, some of which results we explored in the previous assignment. 

This census data counts dwelling units in the city and categorized the population within each unit, aggregating results by community. The original dataset has been partially cleaned for you, removing some columns which are not relevant to the assignment. However, given that there are still 75 columns remaining, you may want to use SQLAlchemy or another bulk load tool to handle the creation of the database table for this dataset, before altering the table as needed to change data types or creating indexes.

The Community Services dataset lists a number of amenities in the city with their address and community. This table has not been altered from the version provided by Open Data Calgary.

## Data cleaning and import

First, import the two CSVs (Community_Services.csv, Census_by_Community_2019.csv) into your own database. You may use what is available to you on datasciencedb or datasciencedb2. You may also create indexes and define keys if appropriate for the column(s) of your choice.

In the section below, you have the option to discuss any data cleaning and wrangling steps performed during this process. This is not a requirement and will not be assessed directly for grading; however, this may help to clarify to your reader exactly what was done, to make your work below more understandable.

In [1]:
# Helper function for making tables look pretty
from prettytable import PrettyTable


def generate_ascii_table(df):
    '''
    Reference: https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
    '''
    x = PrettyTable()
    x.field_names = df.columns.tolist()
    for row in df.values:
        x.add_row(row)
    return x

In [2]:
import pandas as pd
import sqlalchemy as sq

# read in your CSV as a dataframe
census = pd.read_csv("Census_by_Community_2019.csv")
services = pd.read_csv("Community_Services.csv")

In [3]:
print("Length of census table is {0} rows".format(len(census)))
print("Length of services table is {0} rows".format(len(services)))

Length of census table is 306 rows
Length of services table is 206 rows


In [4]:
services_unique = services.nunique()
services_unique[services_unique == len(services)]

Series([], dtype: int64)

In [ ]:
# check if there is any primary key column
census_unique = census.nunique()
census_unique[census_unique == len(census)]

In [ ]:
print("There are {0} unique COMM_CODE values out of {1} rows.".format(services['COMM_CODE'].nunique(), len(services)))

`COMM_CODE` column can be used as a primary key in _census_ table.
And as a foreign key in _services_ table.

In [3]:
# connect to your database; include a cell at the bottom of this notebook to dispose of your engine object
user = 'andrii_voitkiv1'
password = '3RMB68NSY'
host = 'datasciencedb.ucalgary.ca'
database = 'andrii_voitkiv1'

engine = sq.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(user, password, host, database))
# write your dataframe into a table
census.to_sql(name='census', con=engine, if_exists='replace')
services.to_sql(name='services', con=engine, if_exists='replace')

## Part A: Warm-up Questions (10 marks)

Answer the questions below, including any queries you used where necessary. Not all questions will require a SQL query for a correct response. You may wish to use as a source the references which are already provided as part of this notebook.

First, let's look at the Census by Community Data.

1. How many communities are included as part of this census? **(1 mark)**

2. There are 75 columns in the version of the dataset provided to you. While not all of them are equally interesting, briefly describe the following columns. **(3 marks)**

   - CLASS

   - SRG (include in your answer all posslbe values)

   - COMM_STUCTURE

3. What other columns, to your eye, look interesting? Provide an example of a guiding question which would take advantage of one of these columns. **(2 marks)**

In [7]:
# How many communities are included as part of this census?
query0 = '''
SELECT COUNT(*) AS num_of_communities FROM census;
'''

query_table0 = pd.read_sql_query(query0, engine)
generate_ascii_table(query_table0)

num_of_communities
306


_briefly describe the following columns_

- CLASS - this column represents a type of construction, such as Industrial, Major Park Area, Residential, Residual Sub Area;
- SRG - Reflects the yearly development capacity or housing supply as outlined in the Suburban Residential Growth document, the valid values are: BUILT-OUT, DEVELOPING, NON RESIDENTIAL, and N/A
- COMM_STRUCTURE - Used to identify life-cycle patterns and to develop the demographic model of Calgary. Where a decade is listed at least 51% of a community's peak population must be in place by the end of the decade it is assigned to. Valid values are: 1950s, 1960s/1970s, 1970s/1980s, 1980s/1990s, 1990s/2000s, 2000s, Building Out, Centre City, Employment, Inner City, Other, Undeveloped

Reference: https://data.calgary.ca/Demographics/Census-by-Community-2019/rkfr-buzb [Columns in this Dataset]

_What other columns, to your eye, look interesting? Provide an example of a guiding question which would take advantage of one of these columns._


**The guiding question:**
Which communities have the highest/lowest residential vacancy rate (by type of building)?

_Vacant residential building_ - not occupied with the prefix `_VACANT` (excluding manufacturing)
_Vacancy rate_  is the proportion of vacant buildings normalized by number of residents in this community.

**Interpretation:**
The low vacancy rate implies high demand (due to developed infrastructure, good transportation, affordable prices).

Next, let's look at the Community Services table.

4. How many services in total are there? **(1 mark)**


5. List all possible types of community services listed in the table. How many are missing a community code? **(1 mark)**


6. Which column is the most useful as a key to be used in a join? **(1 mark)**


7. Which community has the highest number of available community services? Include the names of any community with this number of services. **(1 mark)**

In [11]:
# How many services in total are there?
query1 = '''
SELECT COUNT(*) AS num_of_services FROM services;
'''

query_table1 = pd.read_sql_query(query1, engine)
generate_ascii_table(query_table1)

num_of_services
206


In [12]:
# List all possible types of community services listed in the table. How many are missing a community code?
query3 = '''
SELECT TYPE, SUM(CASE WHEN COMM_CODE IS NULL then 1 else 0 end) AS count_null
FROM services
GROUP BY 1;
'''

query_table3 = pd.read_sql_query(query3, engine)
generate_ascii_table(query_table3)
# One type of community services which is "Attraction" has four missing community codes.

TYPE,count_null
Attraction,4.0
Community Centre,0.0
Court,0.0
Hospital,0.0
Library,0.0
PHS Clinic,0.0
Social Dev Ctr,0.0
Visitor Info,0.0


_Which column is the most useful as a key to be used in a join?_

`COMM_CODE` is the most useful as a key to be used in a join (in this context when we have census table). We can form one-to-many relationship - one record in census table is related to one or more records services table.
A column `NAME` is a good candidate as a primary key, but it has two duplicates and one of duplicates has the same address. For that reason I wouldn't recommend to use this column as a key.

In [13]:
# Which community has the highest number of available community services?
# Include the names of any community with this number of services.
query4 = '''
SELECT a.COMM_CODE, a.count_services, b.NAME
FROM (SELECT COMM_CODE, COUNT(TYPE) AS count_services
      FROM services
      WHERE COMM_CODE IS NOT NULL
      GROUP BY 1
      ORDER BY count_services DESC
      LIMIT 1) AS a
         JOIN
     (SELECT COMM_CODE, NAME
      FROM census) AS b ON a.COMM_CODE = b.COMM_CODE
'''

query_table4 = pd.read_sql_query(query4, engine)
generate_ascii_table(query_table4)

COMM_CODE,count_services,NAME
DNC,24,DOWNTOWN COMMERCIAL CORE


## Part B: SQL with multiple tables (13 marks)


What is the population of seniors (male and female, or other, aged 65 and over) in the community or communities with the most community services? **(2 marks)**

What is the population of children (male and female, or other, aged 0 to 14), in the community or communities with the most community services?  **(2 marks)**

What is the total population of seniors (male and female, or other, aged 65 and over) in all communities that contain nursing homes?  **(2 marks)**

What is the total number of residents, children (using the definition above) and seniors (using the definitions above) in communities which do not have any community services?  **(2 marks)**

Generate a list of the ten such communities with the greatest number of residents.  **(2 marks)**

Prepare a list of community services as follows: **(3 marks)**
- Each community service should list, for the community which they are in, the total number of residents, the total number of preschool children, and the number of dwellings.
- Community services with no community listed should use a meaningful placeholder in the result.
- Communities with no community services should be included in the results, with a meaningful placeholder where data for community services would ordinarily be included.

In [15]:
# What is the population of seniors (male and female, or other, aged 65 and over) in the community or communities with the most community services?
query5 = '''
SELECT COMM_CODE,
       MF_65_74 + MF_75 + OTHER_65_74 + OTHER_75 AS seniors_over_65
FROM census
WHERE COMM_CODE =
      (SELECT a.COMM_CODE
       FROM (SELECT COMM_CODE, COUNT(TYPE) AS count_services
             FROM services
             WHERE COMM_CODE IS NOT NULL
             GROUP BY 1
             ORDER BY count_services DESC
             LIMIT 1) a);
'''

query_table5 = pd.read_sql_query(query5, engine)
generate_ascii_table(query_table5)

COMM_CODE,seniors_over_65
DNC,820


In [16]:
# What is the population of children (male and female, or other, aged 0 to 14), in the community or communities with the most community services?
query5 = '''
SELECT COMM_CODE,
       MF_0_4 + MF_5_14 + OTHER_0_4 + OTHER_5_14 AS children_under_14
FROM census
WHERE COMM_CODE =
      (SELECT a.COMM_CODE
       FROM (SELECT COMM_CODE, COUNT(TYPE) AS count_services
             FROM services
             WHERE COMM_CODE IS NOT NULL
             GROUP BY 1
             ORDER BY count_services DESC
             LIMIT 1) a);
'''

query_table5 = pd.read_sql_query(query5, engine)
generate_ascii_table(query_table5)

COMM_CODE,children_under_14
DNC,817


In [18]:
# What is the total population of seniors (male and female, or other, aged 65 and over) in all communities that contain nursing homes?
query6 = '''
SELECT SUM(seniors_nursing_homes) AS total_seniors_nursing_homes
FROM (SELECT MF_65_74 + MF_75 + OTHER_65_74 + OTHER_75 AS seniors_nursing_homes
      FROM census
      WHERE NURSING_HM > 0) csnh;
'''
generate_ascii_table(pd.read_sql_query(query6, engine))

total_seniors_nursing_homes
58320.0


In [24]:
# What is the total number of residents, children (using the definition above) and seniors (using the definitions above) in communities which do not have any community services?
'''
Methodology:
1. By left outer joining census with services table (ON COMM_CODE) I get nulls in services.COMM_CODE column,
meaning we don't have some communities from census table listed in services table,
meaning these communities don't have services
2. Filter table from step 1 (WHERE ...)
3. Calculate SUM
'''
query7 = '''
SELECT SUM(temp.population_no_services) AS Total_population_no_services
FROM (SELECT c.MF_65_74 + c.MF_75 + c.OTHER_65_74 + c.OTHER_75 +
             c.MF_0_4 + c.MF_5_14 + c.OTHER_0_4 + c.OTHER_5_14 AS population_no_services
      FROM (SELECT * FROM census) AS c
               LEFT OUTER JOIN
               (SELECT * FROM services) AS s ON c.COMM_CODE = s.COMM_CODE
      WHERE s.COMM_CODE IS NULL) temp
'''

generate_ascii_table(pd.read_sql_query(query7, engine))

Total_population_no_services
156162.0


In [26]:
# Generate a list of the ten such communities with the greatest number of residents
'''
I assume that the word "such" is related to the previous question. And the question sounds:
Generate a list of the ten communities which do not have any community services with the greatest number of residents
'''
query8 = '''
SELECT c.NAME, c.RES_CNT
FROM (SELECT * FROM census) AS c
         LEFT OUTER JOIN
         (SELECT * FROM services) AS s ON c.COMM_CODE = s.COMM_CODE
WHERE s.COMM_CODE IS NULL
ORDER BY c.RES_CNT DESC
LIMIT 10
'''

generate_ascii_table(pd.read_sql_query(query8, engine))


NAME,RES_CNT
PANORAMA HILLS,25710
SADDLE RIDGE,22321
EVERGREEN,21500
CRANSTON,19884
TARADALE,19026
MCKENZIE TOWNE,18283
EVANSTON,17685
COVENTRY HILLS,17667
AUBURN BAY,17607
NEW BRIGHTON,13103


In [34]:
# Prepare a list of community services as follows:
# Each community service should list, for the community which they are in, the total number of residents, the total number of preschool children, and the number of dwellings.

# Community services with no community listed should use a meaningful placeholder in the result

# Communities with no community services should be included in the results, with a meaningful placeholder where data for community services would ordinarily be included

'''
Methodology:
1. census LEFT OUTER JOIN services           - Communities with no community services included
2. census RIGHT JOIN services                - Community services with no community listed
3. table from step 1 UNION table from step 2 - append two tables, duplicates are dropped (we don't need them)
4. IFNULL                                    - function replaces NULL with placeholder
'''

query9 = '''
SELECT IFNULL(temp.comm_name, 'Unknown community name')                               AS community_name,
       IFNULL(temp.TYPE, 'Unknown service type')                                      AS service_type,
       IFNULL(temp.service_name, 'Unknown service name')                              AS service_name,
       IFNULL(temp.RES_CNT, 'Unknown res count served by this service')               AS RES_CNT,
       IFNULL(temp.PRSCH_CHLD, 'Unknown number of prsch chld served by this service') AS PRSCH_CHLD,
       IFNULL(temp.DWELL_CNT, 'Unknown dwell count served by this service')           AS DWELL_CNT
FROM (SELECT c.NAME      AS comm_name,
             s.TYPE,
             s.NAME      AS service_name,
             c.RES_CNT,
             c.PRSCH_CHLD,
             c.DWELL_CNT
      FROM (SELECT * FROM census) AS c
               LEFT OUTER JOIN
               (SELECT * FROM services) AS s ON c.COMM_CODE = s.COMM_CODE

      UNION

      SELECT c.NAME,
             s.TYPE,
             s.NAME,
             c.RES_CNT,
             c.PRSCH_CHLD,
             c.DWELL_CNT
      FROM (SELECT * FROM census) AS c
               RIGHT JOIN
               (SELECT * FROM services) AS s ON c.COMM_CODE = s.COMM_CODE) AS temp;
'''

# generate_ascii_table(pd.read_sql_query(query9, engine).tail(50))
pd.read_sql_query(query9, engine)

,community_name,service_type,service_name,RES_CNT,PRSCH_CHLD,DWELL_CNT
0,CAPITOL HILL,Community Centre,Rosemont Community Centre,4744,325,2440
1,CANADA OLYMPIC PARK,Attraction,WinSport's Canada Olympic Park,0,0,0
2,MIDNAPORE,Community Centre,Mid-Sun Community Centre,7270,442,2898
3,DOWNTOWN COMMERCIAL CORE,Attraction,Arts Commons,8683,503,5679
4,MAPLE RIDGE,Community Centre,Willow Park / Mapleridge Community Centre,1916,72,819
...,...,...,...,...,...,...
376,Unknown community name,Attraction,Spruce Meadows Equestrian Centre,Unknown res count served by this service,Unknown number of prsch chld served by this se...,Unknown dwell count served by this service
377,Unknown community name,Attraction,Peace Bridge,Unknown res count served by this service,Unknown number of prsch chld served by this se...,Unknown dwell count served by this service
378,Unknown community name,Social Dev Ctr,West Area,Unknown res count served by this service,Unknown number of prsch chld served by this se...,Unknown dwell count served by this service
379,Unknown community name,Attraction,The Police Museum,Unknown res count served by this service,Unknown number of prsch chld served by this se...,Unknown dwell count served by this service


## Part C: Evaluating your results  (7 marks)


**Question 1 (3 marks)**

The queries in Part B suggest that there are parts of Calgary which may be over-served by the numbers of community services available, and there are parts of Calgary which may be underserved by kinds of community services which are not available.

Before we continue with this line of inquiry, discuss two factors which may not be captured by either dataset.



Nominate one datset already available from Open Data Calgary which addresses one of your two factors. Remember to reference this using a well-accepted reference format.



_Two factors which may not be captured by either dataset:_
1. Citizens choose a service they'd like to use not only by location but more often by rating, including feedback from other people who used it.

For example, suppose there is a hospital on the other end of the city where the doctors are more qualified. In that case, people will probably opt for that clinic, as concerns about their health are of much high importance than the distance to cover (in case it is not an emergency).

Another example is a long-awaited book that is finally available in a public library outside your community.

The rating of community services can be retrieved from google api.

2. Another factor is a methodology how the services are classified (by communities) with imaginary borders that don't exist in real life. Communities are all of different shape and area. In our dataset, one service is related to one community. But there could possibly be a case when residents of other communities (1,2,3 from picture below) may use this service more often than residents from the community num 4. Perhaps a river or huge park separates them from that community. Alternative way can be to define radius 1, 5, 10 kilometers from service coordinates and check with what communities it intersects. From that, we will get that one service is related to multiple communities.

_Nominate one dataset already available from Open Data Calgary which addresses one of your two factors:_
Census By Community 2017 (includes coordinates of communities). Open Data Calgary. Available from: https://data.calgary.ca/Demographics/Census-By-Community-2017/9h2a-qxc4

<img src="../../../../Desktop/Screenshot 2022-12-04 at 12.21.20.png"/>


**Question 2 (4 marks)**

Imagine that we have stored data relevant to each community in a separate JSON object, each of which has been added to a document in the same collection in MongoDB. Community Services have been added as list of JSON attributes to the relevant community.

For example, we might have
```
{
  Community:"Community Name", 
  ...
  Community_Services:[
      {
          "Community Service": "Community Service Name"
          
      },
      ...
      ]
  }
```
Pick one of the queries from Part B to discuss. Do you think it is more difficult to retrieve the information requested for this query from the pair of relational database tables provided to you, or from a MongoDB collection set up as described? Explain why or why not. 

Let's take this query where we have to find _what is the total number of residents, children and seniors in communities which do not have any community services_?

To solve this with SQL we need JOIN - find all the values in the "census" table, then, using a key COMM_CODE in that table, look for all the matching rows in the "services" table. It is worth mentioning that the "census" table has 70+ columns (not normalized) and must be split into multiple tables. Otherwise, it is suboptimal.

The same query in Mongodb can be done using aggregation ($sum), as MongoDB does not support joins. Our first stage should be a $match that filters for documents that contain a "Community_Services" field and for which the Community_Services array is empty:
db.collection.aggregate([ { $match: { "Community_Services": { $exists: true, $in: [null]} } }, ...

In a group stage, we can aggregate values from multiple documents and perform aggregation operations, such as calculating sum. If we specify a group id value of null, the $group stage returns a single document that aggregates values across all input documents. A good analogy for this process is a **pipeline** when the result from the previous step is an input for the next stage.

Overall, MongoDB is much faster than SQL if the data is indexed appropriately (this is true for all databases). In addition. MongoDB is appropriate for hierarchical data. The more it's nested - the more I'd like to use MongoDB vs SQL.

And to answer the question of which one is more difficult to query, I prefer MongoDB, as related information is stored together, not in different tables. And the process of how you think about the query is more straightforward and feels more natural to how human thinks.

## Part E: Reflection (5 marks)


In 100 to 250 words, identify a concept you have found difficult or confusing from this assignment. Reflect on how your previous learning or experience helped you to understand this concept. Provide your reflection using markdown in the cell below.



In this assignment, I connected a few concepts that seemed distinct to me before. First, the "census" table with 70+ columns made me think about why it is commonly a bad practice to keep a table of this size in a database and how it's related to physical _storage space_. From here, the concept of **normalization** pops up:
1. Find the _primary key_ for efficient and high-performance queries.
2. Convert _types_ of columns (again, to save storage space).
3. Split into _multiple tables_ to avoid information redundancy and optimize storage space.

In my case, there are only two tables, but it could be much more (if I split the census table).
After the steps described above, I'm left with many tables you need to join to retrieve the information asked. And that makes me think about the **set theory** from Discrete algebra combined with implementation **algorithms** (think complexity, BigO) - for loops, hash tables etc.

To conclude, I have to plan carefully how to organize my tables and which keys to use. It will affect future queries' performance while joining and filtering my data. This is especially important when it comes to Big Data.

## References

Both datasets used in this Assignment are licensed under the Open Government License - City of Calgary.

Census by Community 2019 [online], 2022. Open Calgary. Available from: https://data.calgary.ca/Demographics/Census-by-Community-2019/rkfr-buzb [Accessed 26 Nov 2022].

Community Services [online], 2022. Open Calgary. Available from: https://data.calgary.ca/Services-and-Amenities/Community-Services/x34e-bcjz [Accessed 26 Nov 2022].
